# Iceberg ETL Notebook

This notebook demonstrates an ETL pipeline using Apache Spark and Iceberg tables.

## Overview
1. Initialize Spark Session with Iceberg configuration
2. Generate mock sales data
3. Write data to Iceberg table
4. Query and analyze the data
5. Explore Iceberg table features (snapshots, time travel)

## 1. Initialize Spark Session

The Spark session is pre-configured with Iceberg catalog settings via `spark-defaults.conf`:
- Catalog: `demo` (Hive-based Iceberg catalog)
- Warehouse: `s3a://warehouse/` (MinIO)
- Metastore: `thrift://hive-metastore:9083`

In [1]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
spark._jsc.hadoopConfiguration().get("fs.s3a.impl")

'org.apache.hadoop.fs.s3a.S3AFileSystem'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
    TimestampType,
)
from faker import Faker
import random
from datetime import datetime, timedelta

In [3]:
# Create Spark session
# Configuration is loaded from spark-defaults.conf
spark = SparkSession.builder \
    .appName("Iceberg ETL Notebook") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
spark

Spark version: 3.5.5


26/02/23 07:29:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Verify Iceberg catalog is configured
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|        local|
|spark_catalog|
+-------------+



## 2. Define Schema and Generate Mock Data

In [5]:
# Define schema for sales data
schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_name", StringType(), False),
    StructField("customer_email", StringType(), False),
    StructField("customer_city", StringType(), False),
    StructField("customer_country", StringType(), False),
    StructField("product_name", StringType(), False),
    StructField("product_category", StringType(), False),
    StructField("amount", DoubleType(), False),
    StructField("order_date", TimestampType(), False),
    StructField("order_status", StringType(), False),
])

print("Schema defined successfully!")

Schema defined successfully!


In [6]:
def generate_mock_sales_data(num_records=1000):
    """Generate realistic mock sales data using Faker"""
    fake = Faker()
    fake.seed_instance(42)

    data = []
    start_date = datetime(2025, 1, 1)

    for i in range(1, num_records + 1):
        product_categories = ["Electronics", "Clothing", "Books", "Home", "Sports"]
        product_category = random.choice(product_categories)

        # Category-based pricing
        if product_category == "Electronics":
            amount = round(random.uniform(50, 2000), 2)
        elif product_category == "Clothing":
            amount = round(random.uniform(20, 200), 2)
        elif product_category == "Books":
            amount = round(random.uniform(10, 50), 2)
        elif product_category == "Home":
            amount = round(random.uniform(30, 500), 2)
        else:
            amount = round(random.uniform(15, 300), 2)

        days_ago = random.randint(0, 365)

        data.append((
            i,
            fake.name(),
            fake.email(),
            fake.city(),
            fake.country(),
            fake.catch_phrase(),
            product_category,
            amount,
            start_date + timedelta(days=days_ago),
            random.choice(["pending", "completed", "cancelled", "refunded"]),
        ))
    return data

In [7]:
# Generate mock data
print("--- Generating Mock Sales Data ---")
mock_data = generate_mock_sales_data(1000)
df = spark.createDataFrame(mock_data, schema)

print(f"Generated {df.count()} records")
df.printSchema()

--- Generating Mock Sales Data ---


[Stage 0:>                                                        (0 + 16) / 16]

Generated 1000 records
root
 |-- order_id: integer (nullable = false)
 |-- customer_name: string (nullable = false)
 |-- customer_email: string (nullable = false)
 |-- customer_city: string (nullable = false)
 |-- customer_country: string (nullable = false)
 |-- product_name: string (nullable = false)
 |-- product_category: string (nullable = false)
 |-- amount: double (nullable = false)
 |-- order_date: timestamp (nullable = false)
 |-- order_status: string (nullable = false)



In [ ]:
# Preview the data
df.show(10, truncate=True)

## 3. Write Data to Iceberg Table

In [16]:
# Create database if not exists
spark.sql("CREATE DATABASE IF NOT EXISTS demo.sales_db")
spark.sql("SHOW DATABASES IN demo").show()

+-------------+
|    namespace|
+-------------+
|demo.sales_db|
+-------------+



In [17]:
# Write data to Iceberg table
print("--- Writing to Iceberg Table ---")
df.writeTo("demo.sales_db.sales").createOrReplace()

print("Data written successfully!")

--- Writing to Iceberg Table ---


Data written successfully!


## 4. Query and Analyze the Data

In [18]:
# Verify the data
print("--- Total Records ---")
spark.sql("SELECT COUNT(*) as total_records FROM demo.sales_db.sales").show()

--- Total Records ---
+-------------+
|total_records|
+-------------+
|         1000|
+-------------+



In [19]:
# Sales by category
print("--- Sales by Category ---")
spark.sql("""
    SELECT 
        product_category,
        COUNT(*) as num_orders,
        ROUND(SUM(amount), 2) as total_revenue,
        ROUND(AVG(amount), 2) as avg_order_value
    FROM demo.sales_db.sales
    GROUP BY product_category
    ORDER BY total_revenue DESC
""").show()

--- Sales by Category ---


[Stage 9:>                                                          (0 + 4) / 4]

+----------------+----------+-------------+---------------+
|product_category|num_orders|total_revenue|avg_order_value|
+----------------+----------+-------------+---------------+
|     Electronics|       205|    217276.27|        1059.88|
|            Home|       208|     53558.26|         257.49|
|          Sports|       180|     27978.69|         155.44|
|        Clothing|       203|     20396.55|         100.48|
|           Books|       204|      6243.34|           30.6|
+----------------+----------+-------------+---------------+



In [20]:
# Sales by status
print("--- Sales by Status ---")
spark.sql("""
    SELECT 
        order_status,
        COUNT(*) as num_orders,
        ROUND(SUM(amount), 2) as total_amount
    FROM demo.sales_db.sales
    GROUP BY order_status
    ORDER BY num_orders DESC
""").show()

--- Sales by Status ---
+------------+----------+------------+
|order_status|num_orders|total_amount|
+------------+----------+------------+
|   cancelled|       264|    78721.21|
|   completed|       257|     81878.5|
|    refunded|       248|    84431.24|
|     pending|       231|    80422.16|
+------------+----------+------------+



In [21]:
# Top 10 countries by revenue
print("--- Top 10 Countries by Revenue ---")
spark.sql("""
    SELECT 
        customer_country,
        COUNT(*) as num_orders,
        ROUND(SUM(amount), 2) as total_revenue
    FROM demo.sales_db.sales
    GROUP BY customer_country
    ORDER BY total_revenue DESC
    LIMIT 10
""").show()

--- Top 10 Countries by Revenue ---
+--------------------+----------+-------------+
|    customer_country|num_orders|total_revenue|
+--------------------+----------+-------------+
|                Oman|         7|      5692.34|
|       French Guiana|         7|      5576.95|
|Saint Pierre and ...|         4|      5447.18|
|Holy See (Vatican...|         4|      4748.24|
|             Morocco|         9|      4722.49|
|           Nicaragua|         5|      4335.34|
|             Somalia|         5|      4267.92|
|          Cape Verde|         8|      4080.69|
|       Liechtenstein|         5|      4008.78|
|             Burundi|         4|      3950.74|
+--------------------+----------+-------------+



## 5. Explore Iceberg Table Features

In [22]:
# View Iceberg snapshots
print("--- Iceberg Snapshots ---")
spark.sql("SELECT * FROM demo.sales_db.sales.snapshots").show(truncate=False)

--- Iceberg Snapshots ---
+-----------------------+-------------------+---------+---------+----------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                   |summary                                                                                                                                            

In [29]:
# View table history
print("--- Table History ---")
spark.sql("SELECT * FROM demo.sales_db.sales.history").show(truncate=True)

--- Table History ---
+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2026-02-23 06:53:...|6664706160060697349|               NULL|              false|
|2026-02-23 06:56:...|3438309255227403268|               NULL|               true|
|2026-02-23 06:56:...|8488913132314979929|3438309255227403268|               true|
+--------------------+-------------------+-------------------+-------------------+



In [30]:
# View table partitions
print("--- Table Partitions ---")
spark.sql("SELECT * FROM demo.sales_db.sales.partitions").show(truncate=True)

--- Table Partitions ---
+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|        1500|        32|                       220841|                           0|                         0|                           0|                         0|2026-02-23 06:56:...|     8488913132314979929|
+------------+----------+-----------------------------+----------------------------+--------------------------+--------

In [31]:
# View table manifest files
print("--- Manifest Files ---")
spark.sql("SELECT * FROM demo.sales_db.sales.manifests").show(truncate=True)

--- Manifest Files ---
+-------+--------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
|content|                path|length|partition_spec_id|  added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|
+-------+--------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
|      0|$PWD/warehouse/de...| 10049|                0|8488913132314979929|                    16|                        0|                       0|                       0|                          0|                         

## 6. Additional Operations (Optional)

### Append More Data

In [32]:
# Generate additional data and append
print("--- Appending Additional Data ---")
additional_data = generate_mock_sales_data(500)
# Adjust order_id to avoid duplicates
additional_data = [(i + 1000, *rest) for i, *rest in additional_data]
df_additional = spark.createDataFrame(additional_data, schema)

df_additional.writeTo("demo.sales_db.sales").append()

spark.sql("SELECT COUNT(*) as total_records FROM demo.sales_db.sales").show()

--- Appending Additional Data ---
+-------------+
|total_records|
+-------------+
|         2000|
+-------------+



In [33]:
# Check snapshots after append
spark.sql("SELECT * FROM demo.sales_db.sales.snapshots").show(truncate=True)

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2026-02-23 06:53:...|6664706160060697349|               NULL|   append|$PWD/warehouse/de...|{spark.app.id -> ...|
|2026-02-23 06:56:...|3438309255227403268|               NULL|   append|$PWD/warehouse/de...|{spark.app.id -> ...|
|2026-02-23 06:56:...|8488913132314979929|3438309255227403268|   append|$PWD/warehouse/de...|{spark.app.id -> ...|
|2026-02-23 06:57:...| 479311417805473928|8488913132314979929|   append|$PWD/warehouse/de...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+



### Time Travel Query

In [34]:
# Get the first snapshot ID for time travel
snapshots = spark.sql("SELECT snapshot_id FROM demo.sales_db.sales.snapshots ORDER BY committed_at").collect()
if len(snapshots) > 1:
    first_snapshot_id = snapshots[0]["snapshot_id"]
    print(f"--- Querying data at snapshot {first_snapshot_id} (before append) ---")
    spark.sql(f"SELECT COUNT(*) as records_at_first_snapshot FROM demo.sales_db.sales VERSION AS OF {first_snapshot_id}").show()

--- Querying data at snapshot 6664706160060697349 (before append) ---
+-------------------------+
|records_at_first_snapshot|
+-------------------------+
|                     1000|
+-------------------------+



## Cleanup

In [35]:
# Stop Spark session
spark.stop()
print("Spark session stopped.")

Spark session stopped.
